⚠️ *WARNING*

As of 2021-05-26 `abm_plot()` and `abm_video()` [did not work](https://discourse.julialang.org/t/error-no-backend-available-glmakie-cairomakie-wglmakie-when-calling-abm-video-in-interactivedynamics-jl/61626). There might be glitches between `Agents.jl` and `Makie.jl`.

# Example 1 : Schelling's segregation model

This example is taken from `Agents.jl` [tutorial](https://juliadynamics.github.io/Agents.jl/stable/examples/schelling/).

- Agents : They belong to one of two groups (0 or 1).
- Model : Each position of the grid can be occupied by at most one agent.
- For each step
  - If an agent has at least 3 neighbors belonging to the same group, then it is happy.
  - If an agent is unhappy, it keeps moving to new locations until it is happy.

To define an agent type, we should make a mutable struct derived from `AbstractAgent` with 2 mandatory fields:
- `id::Int` . The identifier number of the agent.
- `pos` . For agents on a 2D grid, the position field should be a tuple of 2 integers.

On top of that, we could define other properties for the agents.

```julia
mutable struct SchellingAgent <: AbstractAgent
    id::Int             # The identifier number of the agent
    pos::NTuple{2, Int} # The x, y location of the agent on a 2D grid
    mood::Bool          # whether the agent is happy in its position. (true = happy)
    group::Int          # The group of the agent, determines mood as it interacts with neighbors
end
```

## Setup

First, we create a 2D space with a Chebyshev metric. This leads to *8 neighboring positions* per position (except at the edges of the grid).

In [ ]:
using Agents

# Creating a space
space = GridSpace((10, 10); periodic = false)

We define the Agent type using the [`@agent`](https://juliadynamics.github.io/Agents.jl/stable/api/#@agent-macro) macro. Thus we don't have to setup the mandatory `id` and `pos` fields by ourselves. The relevant fileds are `mood` (whetehr the agent is happy or not) and `group` (which group the agent is on).

In [ ]:
@agent SchellingAgent GridAgent{2} begin
    mood::Bool  # True = happy
    group::Int  # 0 or 1
end

In [ ]:
properties = Dict(:min_to_be_happy => 3) # Parameter for the ABM

schelling = ABM(SchellingAgent, space; properties)

# schelling2 = ABM(SchellingAgent, space; properties = properties, scheduler = Schedulers.by_property(:group),)  # Custom scheduler

We setup the model using an `initialize()` function to make the model easier to reproduce and change its parameter(s).

In [ ]:
using Random # for reproducibility in the RNG

function initialize(; numagents = 320, griddims = (20, 20), min_to_be_happy = 3, seed = 125)
    space = GridSpace(griddims, periodic = false)
    properties = Dict(:min_to_be_happy => min_to_be_happy)
    rng = Random.MersenneTwister(seed)
    model = ABM(
        SchellingAgent, space;
        properties, 
        rng, 
        scheduler = Schedulers.randomly
    )

    # populate the model with agents, adding equal amount of the two types of agents
    # at random positions in the model
    for n in 1:numagents
        agent = SchellingAgent(n, (1, 1), false, n < numagents / 2 ? 1 : 2)
        add_agent_single!(agent, model)  # We don't need to set the starting position. Agents package chooses randomly for us.
    end
    return model
end

Finally, we define a step function `agent_step!()` to determine what happens to each agent. We use some built-in functions: `nearby_agents()` and `move_agent_single!()`.

In [ ]:
function agent_step!(agent, model)
    minhappy = model.min_to_be_happy
    count_neighbors_same_group = 0
    # For each neighbor, get group and compare to current agent's group
    # and increment count_neighbors_same_group as appropriately.
    # Here `nearby_agents` (with default arguments) will provide an iterator
    # over the nearby agents one grid point away, which are at most 8.
    for neighbor in nearby_agents(agent, model)
        if agent.group == neighbor.group
            count_neighbors_same_group += 1
        end
    end
    if count_neighbors_same_group ≥ minhappy
        # The agent is happy
        agent.mood = true
    else
        # Move the agent to a random position
        move_agent_single!(agent, model)  
    end
    return
end

## Stepping the model

In [ ]:
model = initialize()

The `step!()` function moves the model forward. `run!()` is similar to `step!()` but also collects data along the simulation.

In [ ]:
# Move the model by one iteration
step!(model, agent_step!)

In [ ]:
# Move the model by 3 iterations
step!(model, agent_step!, 3)

## Visualization

The `abm_plot()` function visulizes the simulation result. By default `Makie.jl` is used.

In [ ]:
using InteractiveDynamics
using CairoMakie            # Makie with the Cario backend

# Some helper functions to identify agent groups.
groupcolor(a) = a.group == 1 ? :blue : :orange
groupmarker(a) = a.group == 1 ? :circle : :rect

In [ ]:
model = initialize(griddims = (50, 50), numagents = 1800)
figure, _ = abm_plot(model; ac = groupcolor, am = groupmarker, as = 10)
figure  # The static figure

Let's make the simulation move.

In [ ]:
model = initialize(griddims = (50, 50), numagents = 1800)

abm_video(
    "schelling.mp4", model, agent_step!;
    ac = groupcolor, am = groupmarker, as = 10,
    framerate = 4, frames = 20,
    title = "Schelling's segregation model"
)

If you cannot use `Makie.jl`, you can use `plotabm()`, which uses `Plots.jl` to render the results.

In [ ]:
# Alternative visualization using Plots.jl
# This will emmit a lot of warnings. Just ignore them.

using Plots

model = initialize(griddims = (50, 50), numagents = 1800)

anim = @animate for i in 1:25
    step!(model, agent_step!)
    plotabm(model; warn = false, ac = groupcolor, am = groupmarker, as = 5, 
        aspect_ratio=:equal, xlims=(0.0, 50.5), ylims = (0.0, 50.5), 
        title = "Schelling: Step $i", size=(600, 600))
end

mp4(anim, fps = 5)

## Data analysis

The `run!()` function collects data in the `DataFrames` format.

In [ ]:
# aggregated data. fields we want to extract
adata = [:pos, :mood, :group]

model = initialize()
data, _ = run!(model, agent_step!, 5; adata)
data[1:10, :] # print only the first 10 rows

In [ ]:
# adata also accepts functions

x(agent) = agent.pos[1]
model = initialize()
adata = [x, :mood, :group]
data, _ = run!(model, agent_step!, 5; adata)
data[1:10, :]

## Launching an interactive app

See [this section](https://juliadynamics.github.io/Agents.jl/stable/examples/schelling/#Launching-the-interactive-application) using `abm_data_exploration()` in the official tutorial.

## Saving the model state

- `AgentsIO.save_checkpoint()`
- `AgentsIO.load_checkpoint()`

In [ ]:
model = initialize(numagents = 200, min_to_be_happy = 5, seed = 42)

run!(model, agent_step!, 40)

figure, _ = abm_plot(model; ac = groupcolor, am = groupmarker, as = 10)
# State after 40 steps
figure

In [ ]:
# Save the model state (to be loaded later)
# jld2 as to Julia is "pickle" as to Python
AgentsIO.save_checkpoint("schelling.jld2", model)

In [ ]:
# Load the model
model = AgentsIO.load_checkpoint("schelling.jld2"; scheduler = Schedulers.randomly)

In [ ]:
# Mees with this model by adding more blue agents
for i in 1:100
    agent = SchellingAgent(nextid(model), (1, 1), false, 1)
    add_agent_single!(agent, model)
end

In [ ]:
figure, _ = abm_plot(model; ac = groupcolor, am = groupmarker, as = 10)
figure

In [ ]:
step!(model, agent_step!, 40)

figure, _ = abm_plot(model; ac = groupcolor, am = groupmarker, as = 10)
figure

And let's try what happens if we add one more group (green) of agents.

In [ ]:
model = AgentsIO.load_checkpoint("schelling.jld2"; scheduler = Schedulers.randomly)

for i in 1:100
    agent = SchellingAgent(nextid(model), (1, 1), false, 3)
    add_agent_single!(agent, model)
end

groupcolor(a) = (:blue, :orange, :green)[a.group]
groupmarker(a) = (:circle, :rect, :cross)[a.group]

figure, _ = abm_plot(model; ac = groupcolor, am = groupmarker, as = 10)
figure

In [ ]:
step!(model, agent_step!, 40)

figure, _ = abm_plot(model; ac = groupcolor, am = groupmarker, as = 10)
figure